# Estimating local and global feature importance scores using DiCE

Summaries of counterfactual examples can be used to estimate importance of features. Intuitively, a feature that is changed more often to generate a proximal counterfactual is an important feature. We use this intuition to build a feature importance score. 

This score can be interpreted as a measure of the **necessity** of a feature to cause a particular model output. That is, if the feature's value changes, then it is likely that the model's output class will also change (or the model's output will significantly change in case of regression model).  

Below we show how counterfactuals can be used to provide local feature importance scores for any input, and how those scores can be combined to yield a global importance score for each feature.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from lightgbm import LGBMClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Preliminaries: Loading the data and ML model

In [3]:
from dice_ml.utils import helpers # helper functions
dataset = helpers.load_adult_income_dataset()
helpers.get_adult_data_info()

{'age': 'age',
 'workclass': 'type of industry (Government, Other/Unknown, Private, Self-Employed)',
 'education': 'education level (Assoc, Bachelors, Doctorate, HS-grad, Masters, Prof-school, School, Some-college)',
 'marital_status': 'marital status (Divorced, Married, Separated, Single, Widowed)',
 'occupation': 'occupation (Blue-Collar, Other/Unknown, Professional, Sales, Service, White-Collar)',
 'race': 'white or other race?',
 'gender': 'male or female?',
 'hours_per_week': 'total work hours per week',
 'income': '0 (<=50K) vs 1 (>50K)'}

In [4]:
target = dataset["income"]
# Split data into train and test
from sklearn.model_selection import train_test_split
datasetX = dataset.drop("income", axis=1)
x_train, x_test, y_train, y_test = train_test_split(datasetX, 
                                                    target, 
                                                    test_size = 0.2,
                                                    random_state=0,
                                                    stratify=target)

numerical=["age", "hours_per_week"]
categorical = x_train.columns.difference(numerical) 
from sklearn.compose import ColumnTransformer

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical),
        ('cat', categorical_transformer, categorical)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', LGBMClassifier())])


In [5]:
import dice_ml
from dice_ml import Dice

d = dice_ml.Data(dataframe=dataset, continuous_features=['age', 'hours_per_week'], outcome_name='income')
model = clf.fit(x_train, y_train)
m = dice_ml.Model(model=model, backend="sklearn")

## Local feature importance

We first generate counterfactuals for a given input point. 

In [6]:
exp = Dice(d, m, method="random")
query_instance = x_train[2:3]
e1 = exp.generate_counterfactuals(query_instance, total_CFs=2)
e1.visualize_as_dataframe(show_only_changes=True)

Diverse Counterfactuals found! total time taken: 00 min 00 sec
Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,31,Private,Some-college,Divorced,Blue-Collar,White,Male,40,0.035



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,50.0,-,Assoc,Married,Professional,-,Female,-,0.605
1,39.0,-,Assoc,Married,Professional,-,Female,85.0,0.671


These can now be used to calculate the feature importance scores. 

In [7]:
imp = exp.feature_importance(query_instance, cf_examples_list=e1.cf_examples_list)
print(imp.local_importance)

[{'workclass': 0.0, 'education': 1.0, 'marital_status': 1.0, 'occupation': 1.0, 'race': 0.0, 'gender': 1.0, 'age': 1.0, 'hours_per_week': 0.5}]


Feature importance can also be estimated directly, by leaving the `cf_examples_list` argument blank.

In [8]:
imp = exp.feature_importance(query_instance, posthoc_sparsity_param=None)
print(imp.local_importance)

Diverse Counterfactuals found! total time taken: 00 min 00 sec
[{'workclass': 0.8, 'education': 0.7, 'marital_status': 1.0, 'occupation': 0.9, 'race': 0.5, 'gender': 0.5, 'age': 1.0, 'hours_per_week': 1.0}]


## Global importance

For global importance, we need to generate counterfactuals for a representative sample of the dataset. 

In [13]:
cobj=exp.feature_importance(x_train[0:10], total_CFs=2, posthoc_sparsity_param=None)
print(cobj.summary_importance)

Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec
{'workclass': 0.35, 'education': 1.0, 'marital_status': 0.7, 'occupation': 0.95, 'race': 0.2, 'gender': 0.8, 'age': 1.0, 'hours_per_week': 1.0}


## Convert the counterfactual output to json

In [14]:
json_str = cobj.to_json()
print(json_str)

{
  "cf_examples_list": [
    "{\"data_interface\": {\"outcome_name\": \"income\", \"data_df\": \"dummy_data\"}, \"model_type\": \"classifier\", \"desired_class\": 1.0, \"desired_range\": null, \"test_instance_df\": \"{\\\"age\\\":{\\\"0\\\":38},\\\"workclass\\\":{\\\"0\\\":\\\"Private\\\"},\\\"education\\\":{\\\"0\\\":\\\"HS-grad\\\"},\\\"marital_status\\\":{\\\"0\\\":\\\"Married\\\"},\\\"occupation\\\":{\\\"0\\\":\\\"Blue-Collar\\\"},\\\"race\\\":{\\\"0\\\":\\\"White\\\"},\\\"gender\\\":{\\\"0\\\":\\\"Male\\\"},\\\"hours_per_week\\\":{\\\"0\\\":44},\\\"income\\\":{\\\"0\\\":0.367}}\", \"final_cfs_df\": \"{\\\"age\\\":{\\\"0\\\":50.0,\\\"1\\\":39.0},\\\"workclass\\\":{\\\"0\\\":\\\"Private\\\",\\\"1\\\":\\\"Private\\\"},\\\"education\\\":{\\\"0\\\":\\\"Assoc\\\",\\\"1\\\":\\\"Assoc\\\"},\\\"marital_status\\\":{\\\"0\\\":\\\"Married\\\",\\\"1\\\":\\\"Married\\\"},\\\"occupation\\\":{\\\"0\\\":\\\"Professional\\\",\\\"1\\\":\\\"Professional\\\"},\\\"race\\\":{\\\"0\\\":\\\"White\\\",\\\

## Convert the json output to a counterfactual object

In [16]:
imp_r=imp.from_json(json_str)
print([o for o in imp_r.cf_examples_list])
print(imp_r.local_importance)
print(imp_r.summary_importance)

[<dice_ml.diverse_counterfactuals.CounterfactualExamples object at 0x7f23393ea040>, <dice_ml.diverse_counterfactuals.CounterfactualExamples object at 0x7f23393ead00>, <dice_ml.diverse_counterfactuals.CounterfactualExamples object at 0x7f233ab4c430>, <dice_ml.diverse_counterfactuals.CounterfactualExamples object at 0x7f233a413e50>, <dice_ml.diverse_counterfactuals.CounterfactualExamples object at 0x7f233a9287f0>, <dice_ml.diverse_counterfactuals.CounterfactualExamples object at 0x7f233a928c10>, <dice_ml.diverse_counterfactuals.CounterfactualExamples object at 0x7f233a928f70>, <dice_ml.diverse_counterfactuals.CounterfactualExamples object at 0x7f233a928f40>, <dice_ml.diverse_counterfactuals.CounterfactualExamples object at 0x7f233a16dd00>, <dice_ml.diverse_counterfactuals.CounterfactualExamples object at 0x7f233a16d610>]
[{'workclass': 0.0, 'education': 1.0, 'marital_status': 0.0, 'occupation': 1.0, 'race': 0.0, 'gender': 1.0, 'age': 1.0, 'hours_per_week': 1.0}, {'workclass': 1.0, 'educa